In [44]:
# import os
# import numpy as np
# from skimage import io
# from sklearn.model_selection import train_test_split
# from tensorflow.keras.utils import to_categorical
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from PIL import Image
# import pandas as pd

In [45]:
# image_folder = 'patch_images'
# csv_file = 'data_labels_mainData.csv'

In [46]:
# cwd = os.getcwd()

In [47]:
# image_folder_path = os.path.join(cwd, image_folder)
# csv_file_path = os.path.join(cwd, csv_file)

In [48]:
# df = pd.read_csv(csv_file_path)

In [49]:
# images = []
# for index, row in df.iterrows():
#     image_filename = row['ImageName']
#     image_filepath = os.path.join(image_folder_path, image_filename)
    
#     try:
#         image = Image.open(image_filepath)
        
#         image.load()
        
#         images.append(image)
        
#     except Exception as e:
#         print(f"Error loading image {image_filepath}: {e}")

# len(images)


### RGb 

In [50]:
import os
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout

# Step 1: Read the CSV file and create a dataframe
csv_file = "data_labels_mainData.csv"
data = pd.read_csv(csv_file)

# Step 2: Read the images from the directory and convert them to arrays
image_dir = "patch_images"
image_arrays = []
for image_name in data["ImageName"]:
    image_path = os.path.join(image_dir, image_name)
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # convert to RGB format
    img = cv2.resize(img, (224, 224)) # resize the image to 224 x 224
    image_arrays.append(img)
image_arrays = np.array(image_arrays)

# Step 3: Merge the image arrays with the CSV dataframe
data["image_data"] = list(image_arrays)
data = data.drop("ImageName", axis=1)

# Step 4: Split the data into training and testing sets
X = data.drop("isCancerous", axis=1)
y = data["isCancerous"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 5: Build and train a model using the merged dataset
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(np.stack(X_train["image_data"]), y_train, epochs=10, batch_size=32)

# Step 6: Use the trained model to predict the isCancerous label of new images



Epoch 1/10
248/248 [==============================] - 211s 849ms/step - loss: 34.3127 - accuracy: 0.5790
Epoch 2/10
248/248 [==============================] - 211s 850ms/step - loss: 0.6839 - accuracy: 0.5846
Epoch 3/10
248/248 [==============================] - 216s 869ms/step - loss: 0.6806 - accuracy: 0.5846
Epoch 4/10
248/248 [==============================] - 212s 855ms/step - loss: 0.6794 - accuracy: 0.5846
Epoch 5/10
248/248 [==============================] - 209s 842ms/step - loss: 0.6790 - accuracy: 0.5846
Epoch 6/10
248/248 [==============================] - 207s 835ms/step - loss: 0.6789 - accuracy: 0.5846
Epoch 7/10
248/248 [==============================] - 205s 827ms/step - loss: 0.6788 - accuracy: 0.5846
Epoch 8/10
248/248 [==============================] - 206s 830ms/step - loss: 0.6788 - accuracy: 0.5846
Epoch 9/10
248/248 [==============================] - 213s 858ms/step - loss: 0.6788 - accuracy: 0.5846
Epoch 10/10
248/248 [==============================] - 208s 840

In [51]:
new_image_path = "patch_images/22405.png"
new_image = cv2.imread(new_image_path)
new_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB)
new_image = cv2.resize(new_image, (224, 224))
new_image_array = np.array([new_image])
prediction = model.predict(new_image_array)
if prediction[0][0] > 0.5:
    print("The image is cancerous.")
else:
    print("The image is not cancerous.")

1/1 [==============================] - 0s 66ms/step
The image is not cancerous.


In [52]:
# Step 6: Make predictions on the test data
y_pred = model.predict(np.stack(X_test["image_data"]))
y_pred = (y_pred > 0.5).astype(int)

# Step 7: Evaluate the model's performance on the test data
accuracy = (y_pred == y_test.values.reshape(-1,1)).mean()
print("Accuracy on test set:", accuracy)

62/62 [==============================] - 10s 155ms/step
Accuracy on test set: 0.6005050505050505


## Using grayscale

In [2]:
import os
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split, GridSearchCV
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.wrappers.scikit_learn import KerasClassifier

# Step 1: Read the CSV file and create a dataframe
csv_file = "data_labels_mainData.csv"
data = pd.read_csv(csv_file)

# Step 2: Read the images from the directory and convert them to arrays
image_dir = "patch_images"
image_arrays = []
for image_name in data["ImageName"]:
    image_path = os.path.join(image_dir, image_name)
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # convert to grayscale format
    img = cv2.resize(img, (224, 224))
    image_arrays.append(img)
image_arrays = np.array(image_arrays)

In [3]:
data["image_data"] = list(image_arrays)
data = data.drop("ImageName", axis=1)

# Step 4: Split the data into training and testing sets
X = data.drop("isCancerous", axis=1)
y = data["isCancerous"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
# Step 5: Define the model
def create_model(conv1_filters=32, conv2_filters=64, dense_units=128, dropout_rate=0.5):
    model = Sequential()
    model.add(Conv2D(conv1_filters, kernel_size=(3, 3), activation="relu", input_shape=(224, 224, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(conv2_filters, kernel_size=(3, 3), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(dense_units, activation="relu"))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model


# Step 6: Define the hyperparameters to tune and perform the search
param_grid = {
    "conv1_filters": [32, 64],
    "conv2_filters": [64, 128],
    "dense_units": [128, 256],
    "dropout_rate": [0.3, 0.5],
}
model = KerasClassifier(
    build_fn=create_model,
    epochs=10,
    batch_size=32,
    verbose=2
)
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=3,
    n_jobs=-1,
    verbose=2,
    scoring="accuracy",
)
grid_search.fit(np.stack(X_train["image_data"]), y_train)

# Step 7: Print the best parameters and accuracy score
print("Best parameters:", grid_search.best_params_)
print("Accuracy score:", grid_search.best_score_)

C:\Users\sumet\AppData\Local\Temp\ipykernel_7184\1269362714.py:23: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(


Fitting 3 folds for each of 16 candidates, totalling 48 fits


TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.


In [58]:

# Step 5: Make predictions on a new image
new_image_path = "patch_images/4850.png"
new_image = cv2.resize(cv2.imread(new_image_path, cv2.IMREAD_GRAYSCALE), (224, 224)) / 255.0
new_image_array = np.array([new_image])
prediction = grid_search.predict(new_image_array)
if prediction[0] > 0.5:
    print("The image is cancerous.")
else:
    print("The image is not cancerous.")


1/1 [==============================] - 0s 27ms/step
The image is cancerous.


In [59]:
y_pred = grid_search.predict(np.stack(X_test["image_data"]))
y_pred = (y_pred > 0.5).astype(int)

# Step 7: Evaluate the model's performance on the test data
accuracy = (y_pred == y_test.values.reshape(-1,1)).mean()
print("Accuracy on test set:", accuracy)

62/62 [==============================] - 0s 4ms/step
Accuracy on test set: 0.7287878787878788
